In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

from torch.autograd import Variable
from torchvision import datasets, models, transforms

In [2]:
import numpy as np
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [5]:
data_dir = 'data'

In [6]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}

In [7]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'test']}

In [8]:
dataset_sizes  = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 793, 'test': 207}

In [9]:
class_names = image_datasets['train'].classes
class_names

['cavalo',
 'comida',
 'dinossauro',
 'elefante',
 'flor',
 'indio',
 'montanha',
 'monumento',
 'onibus',
 'praia']

In [10]:
model = models.resnet18(pretrained=True)

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [13]:
def train(model, dataloader, optimizer, criterion, epoch):
    model.train()
    
    for batch_idx, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        prediction = model(x)

        loss = criterion(prediction, y)
        loss.backward()

        optimizer.step() 
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.data.item()))

In [14]:
def val(model, dataloader, criterion):
    model.eval()
    
    val_loss = 0
    correct = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            
            prediction = model(x)
            val_loss += criterion(prediction, y).item()
            
            y_hat = prediction.max(1, keepdim=True)[1]
            correct += y_hat.eq(y.view_as(y_hat)).sum().item()
    
    val_loss /= len(dataloader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(dataloader.dataset),
        100. * correct / len(dataloader.dataset)))

In [15]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model.to(device);

In [16]:
for epoch in range(1, 30+1):
    train(model, dataloaders['train'], optimizer, criterion, epoch)
    val(model, dataloaders['test'], criterion)
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.9

Train Epoch: 1 [0/793 (0%)]	Loss: 2.302897
Train Epoch: 1 [640/793 (80%)]	Loss: 2.058257

Test set: Average loss: 0.7312, Accuracy: 39/207 (19%)

Train Epoch: 2 [0/793 (0%)]	Loss: 1.785908
Train Epoch: 2 [640/793 (80%)]	Loss: 1.154475

Test set: Average loss: 0.0492, Accuracy: 105/207 (51%)

Train Epoch: 3 [0/793 (0%)]	Loss: 1.189389
Train Epoch: 3 [640/793 (80%)]	Loss: 1.110614

Test set: Average loss: 0.0585, Accuracy: 88/207 (43%)

Train Epoch: 4 [0/793 (0%)]	Loss: 1.170841
Train Epoch: 4 [640/793 (80%)]	Loss: 1.136728

Test set: Average loss: 0.0615, Accuracy: 102/207 (49%)

Train Epoch: 5 [0/793 (0%)]	Loss: 1.476130
Train Epoch: 5 [640/793 (80%)]	Loss: 0.897997

Test set: Average loss: 0.0333, Accuracy: 137/207 (66%)

Train Epoch: 6 [0/793 (0%)]	Loss: 1.244905
Train Epoch: 6 [640/793 (80%)]	Loss: 1.020125

Test set: Average loss: 0.0324, Accuracy: 135/207 (65%)

Train Epoch: 7 [0/793 (0%)]	Loss: 0.806854
Train Epoch: 7 [640/793 (80%)]	Loss: 1.468422

Test set: Average loss: 0.0290

In [17]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [20]:
torch.save(model, './models/resnet18')

In [30]:
model = torch.load('./models/resnet18')

In [31]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [32]:
model = models.resnet18(pretrained=False)
model.fc = Identity()

In [33]:
torch.save(model, './models/resnet18-encoder')

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Identity. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [34]:
!ls data

featv1_ANSI.csv  test  train


In [35]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 793
     Root Location: data/train
     Transforms (if any): Compose(
                              RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                              RandomHorizontalFlip(p=0.5)
                              RandomRotation(degrees=(-30, 30), resample=False, expand=False)
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                          )
     Target Transforms (if any): None, 'test': Dataset ImageFolder
     Number of datapoints: 207
     Root Location: data/test
     Transforms (if any): Compose(
                              Resize(size=224, interpolation=PIL.Image.BILINEAR)
                              CenterCrop(size=(224, 224))
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0

In [52]:
from PIL import Image
import math

In [53]:
images = !ls data/images

In [54]:
def name_to_num(name):
    return int(name[:-4])

In [103]:
all_images = []
for image in images:
    image_data = Image.open('data/images/{}'.format(image))
    image_tensor = data_transforms['test'](image_data)
    image_tensor.unsqueeze_(0)
    
    features = model(image_tensor)
    
    label = int(math.floor(name_to_num(image)/100))
    temp_dict = {i:v for i, v in enumerate(features.data.numpy().ravel())}
    temp_dict['Class'] = label
    
    all_images.append(temp_dict)

In [104]:
import pandas as pd

In [105]:
all_images_df = pd.DataFrame(data=all_images)

In [106]:
all_images_df.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,Class
0,0.862178,0.839308,0.784739,0.874484,0.801640,0.824449,0.823280,0.761248,0.835101,0.759689,...,0.703386,0.770972,0.864313,0.757855,0.822150,0.826606,0.803982,0.754933,0.887595,0
1,0.888839,0.825846,0.789228,0.872175,0.764380,0.907626,0.844091,0.827957,0.855548,0.712574,...,0.706060,0.793318,0.845691,0.907800,0.779318,0.901443,0.842881,0.920459,0.867823,1
2,0.863721,0.761275,0.742764,0.711046,0.848023,0.894249,0.962257,0.758384,0.819211,0.757989,...,0.782618,0.730807,0.777283,0.838118,0.934164,0.765030,0.840798,0.787319,0.751451,1
3,0.915312,0.865700,0.803962,0.780047,0.814519,0.810747,0.870463,0.927868,0.827903,0.790925,...,0.696763,0.759936,0.811648,0.798482,0.729883,0.774720,0.811142,0.691062,0.738499,1
4,0.848264,0.728891,0.748240,0.850766,0.746471,0.801984,0.765300,0.806117,0.766603,0.881900,...,0.781184,0.842300,0.769603,0.865306,0.823036,0.942358,0.743737,0.729783,0.820932,1


In [107]:
all_images_df.to_csv('featv2_ANSI.csv', index=False)